Uses this data set: https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions

And also some randomly generated info

In [1]:
import pandas as pd
import numpy as np
from personal_filepaths import *
import ast
from sqlalchemy import create_engine
from credentials import username, password
from models import *
from sqlalchemy.orm import sessionmaker
from sqlalchemy import event
import random

pd.options.display.max_columns = 20
np.random.seed(42)

In [2]:
# For importing all data
users_df = pd.read_csv(PP_users)
recipes_df = pd.read_csv(PP_recipes)
interactions_df = pd.read_csv(RAW_interactions)
recipes_info_df = pd.read_csv(RAW_recipes)
ingredients_df = pd.read_pickle(ingr_map)
user_pass_df = pd.read_csv(user_pass)

In [3]:
# Drop unneccesary info
users_df.drop(['techniques', 'n_items', 'n_ratings'], axis=1, inplace=True)
recipes_df.drop(['i', 'name_tokens', 'techniques', 'calorie_level', 'ingredient_tokens', 'steps_tokens'], axis=1, inplace=True)
interactions_df.drop(['review'], axis=1, inplace=True)
recipes_info_df.drop(['n_steps', 'n_ingredients', 'ingredients', 'nutrition'], axis=1, inplace=True)
ingredients_df.drop(['raw_ingr', 'raw_words', 'processed', 'len_proc', 'count'], axis=1, inplace=True)

In [4]:
# Fill in user data and adjust tables accordingly, along with some other processing

# This section fills in the fake usernames and passwords
users_df = users_df.head(10000)
users_df['username'] = user_pass_df['Username']
users_df['password'] = user_pass_df['Password']

# This section removes any data that references users we deleted
recipes_info_df = recipes_info_df.loc[recipes_info_df['contributor_id'].isin(users_df['u'])]
recipes_df = recipes_df.loc[recipes_df['id'].isin(recipes_info_df['id'])]
interactions_df = interactions_df.loc[interactions_df['user_id'].isin(users_df['u'])]
interactions_df = interactions_df.loc[interactions_df['recipe_id'].isin(recipes_info_df['id'])]

# This just removes unnecessary duplicate data
ingredients_df.drop_duplicates(inplace=True)

# This generates fake account creation and login dates
users_df['creation_date'] = np.random.choice(pd.date_range('1999-08-06', '2021-03-01'), len(users_df))
users_df['last_access_date'] = np.random.choice(pd.date_range('2021-03-01', '2021-03-28'), len(users_df))

# Generate fake serving and difficulty data
recipes_info_df['servings'] = np.random.randint(1, 10, len(recipes_info_df))
recipes_info_df['difficulty'] = recipes_info_df['servings'].apply(lambda x: random.choice(['easy', 'easy_medium', 'medium', 'medium_hard', 'hard']))

# Generate fake aisle number data
ingredients_df['aisle'] = np.random.randint(1, 20, len(ingredients_df))

# Generate fake scale data
interactions_df['scale'] = np.random.randint(1, 5, len(interactions_df))

In [5]:
# Generate tables of data based on the data that already exists

# Generate the steps table data
ids = []
step_nums = []
steps = []

for index, row in recipes_info_df.iterrows():
    step_num = 1
    for step in ast.literal_eval(row['steps']):
        ids.append(row['id'])
        step_nums.append(step_num)
        steps.append(step)
        step_num += 1

steps_df = pd.DataFrame(list(zip(ids, step_nums, steps)), columns=['id', 'step_nr', 'instruction'])

# Generate the categories table data
ids = []
categories = []

for index, row in recipes_info_df.iterrows():
    for tag in ast.literal_eval(row['tags']):
        ids.append(row['id'])
        categories.append(tag)

categories_df = pd.DataFrame(list(zip(ids, categories)), columns=['id', 'category'])

# Generate the recipe ingredient data
recipe_ids = []
ing_ids = []

for index, row in recipes_df.iterrows():
    for ingredient in ast.literal_eval(row['ingredient_ids']):
        recipe_ids.append(row['id'])
        ing_ids.append(int(ingredient))

recipe_ingredients_df = pd.DataFrame(list(zip(recipe_ids, ing_ids)), columns=['recipe_id', 'ingredient_id'])
recipe_ingredients_df['amount'] = np.random.randint(1, 10, len(recipe_ingredients_df))

In [6]:
# Generate the user pantries, most of which is random
pantry_id = []
ing_id = []
user_id = []
expiration_date = []
purchase_date = []
quantity_bought = []

count = 1

for index, row in users_df.iterrows():
    for index2, row2 in ingredients_df.sample(n=100, replace=False).iterrows():
        pantry_id.append(count)
        ing_id.append(row2['id'])
        user_id.append(row['u'])
        count += 1

pantry_df = pd.DataFrame(list(zip(pantry_id, ing_id, user_id)), columns=['pantry_item_id', 'ingredient_id', 'user_id'])
pantry_df['purchase_date'] = np.random.choice(pd.date_range('2021-03-10', '2021-03-28'), len(pantry_df))
pantry_df['expiration_date'] = pantry_df['purchase_date'] + pd.DateOffset(days=14)
pantry_df['quantity_bought'] = np.random.randint(10, 20, len(pantry_df))
pantry_df['current_quantity'] = pantry_df['quantity_bought']

In [8]:
# Rename tables to fall in line with existing tables

users_df.rename(columns={"u":"id"}, inplace=True)
recipes_info_df.rename(columns={"contributor_id":"created_by", "minutes":"cook_time", "submitted":"creation_date"}, inplace=True)
categories_df.rename(columns={"id":"recipe_id", "category":"category_type"}, inplace=True)
interactions_df.rename(columns={"date":"cook_date"}, inplace=True)
ingredients_df.rename(columns={"replaced":"name"}, inplace=True)
# No renaming for pantry_df
# no renaming for recipe_ingredients_df
steps_df.rename(columns={"id":"recipe_id"}, inplace=True)

In [9]:
users_df.drop(['items', 'ratings'], axis=1, inplace=True)
recipes_info_df.drop(['steps', 'tags'], axis=1, inplace=True)

In [29]:
recipes_info_df['difficulty'] = recipes_info_df['servings'].apply(lambda x: random.choice(['easy', 'easy_medium', 'medium', 'medium_hard', 'hard']))

In [52]:
recipe_ingredients_df.drop_duplicates(['recipe_id', 'ingredient_id'], inplace=True)

In [11]:
db_string = "postgresql+psycopg2://" + username + ":" + password + "@reddwarf.cs.rit.edu:5432/p320_03d"
engine = create_engine(db_string)

In [34]:
users_df.to_sql('users', con=engine, index=False, if_exists='append', method='multi')
recipes_info_df.to_sql('recipes', con=engine, index=False, if_exists='append', method='multi')
ingredients_df.to_sql('ingredients', con=engine, index=False, if_exists='append', method='multi')
categories_df.to_sql('categories', con=engine, index=False, if_exists='append', method='multi')
interactions_df.to_sql('cookedby', con=engine, index=False, if_exists='append', method='multi')
pantry_df.to_sql('pantry', con=engine, index=False, if_exists='append', method='multi')
recipe_ingredients_df.to_sql('recipe_ingredients', con=engine, index=False, if_exists='append', method='multi')
steps_df.to_sql('steps', con=engine, index=False, if_exists='append', method='multi')